Analisis de errores:

1. En que se equivocan. 
  Lista de id, etiqueta, predicho, real, error
2. Ganancia de de información de los datasets

3. Distribucion de las palabras por frecuencia

In [1]:
#Cargar lista de documentos y agregarlos a un objeto de pandas
import sys
sys.path.append('D:/v1ktop/Drive/REPOS/augmentation_ap/')

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from preprocesing.load_datasets import Dataset
from classifier.SVM_Text import SVM_text

In [3]:
 %load_ext autoreload

In [4]:
 %autoreload 2

## Conjuntos de datos

In [5]:
dataset_key="erisk18_test"
data_cls=Dataset(key=dataset_key, remove_end=True)

In [10]:
training=data_cls.get_dataset( folder_name="prep_chunks", truth_name="train_golden_truth.txt",
                              return_id=True, partition="training", chunks=False)

In [11]:
len(training)

3

In [14]:
training[2][0]

'trainsubject1015'

In [16]:
test = data_cls.get_dataset( folder_name="prep_chunks", truth_name="test_golden_truth.txt",
                              return_id=True, partition="test", chunks=False)

In [17]:
len(test)

3

In [18]:
test[2][0]

'subject1010'

SVM

Base
0
0
0.52
0.78
0.39
 
L0
1
10
0.54
0.80
0.41
 
L1
5
10
0.54
0.59
0.51
 
Over
2
0
0.53
0.51
0.56
 
Thesaurus
10
0
0.52
0.50
0.53
 
Xi
4
10
0.54
0.57
0.52

SVM-C
Base
0
0
0.53
0.41
0.72
 
L0
1
20
0.44
0.77
0.30
 
L1
1
30
0.54
0.57
0.51
 
Over
8
0
0.53
0.48
0.58
 
Thesaurus
3
0
0.53
0.51
0.54
 
Xi
6
10
0.55
0.59
0.52


## Clasificador

In [63]:
svm= SVM_text(training[0],training[1], weighted=True)

In [64]:
svm.extract_features(test[0], feature="word", method="tf-idf", nrange1=(1,1) , nrange2=None,
                     k=None, stop_words="english", norm="l2", idf=False, feature_selection=False)


# Features:  (486, 136467)
Training:  (486, 136467)
Testing:  (820, 136467)


In [65]:
score= svm.train_and_test(test[1] , p_label="1")

Confusion matrix:  [[660  81]
 [ 22  57]]
Scores: [136467, 0.874390243902439, 0.41304347826086957, 0.7215189873417721, 0.5253456221198157]


In [66]:
import pandas as pd

In [67]:
def show_results(ids, truths, predicted):
    df = pd.DataFrame(truths,index=ids, columns=["truth"])
    df["predicted"]=predicted
    df["error"]=df["truth"]-df["predicted"]
    return df

In [68]:
dic_docs=dict(zip(test[2], test[0]))

In [69]:
false_p=show_results(test[2], test[1], svm.predicted)

In [70]:
false_p[false_p["error"]<0]

,truth,predicted,error
subject1055,0,1,-1
subject1166,0,1,-1
subject1178,0,1,-1
subject1258,0,1,-1
subject1351,0,1,-1
...,...,...,...
subject9822,0,1,-1
subject9832,0,1,-1
subject9877,0,1,-1
subject9880,0,1,-1


In [71]:
false_pos_all= pd.DataFrame(test[1],index=test[2], columns=["truth"])

In [72]:
false_pos_all["base"]=svm.predicted

In [73]:
false_pos_all

,truth,base
subject1010,0,0
subject1015,0,0
subject1044,0,0
subject1055,0,1
subject1073,0,0
...,...,...
subject9949,0,0
subject9951,1,0
subject9958,0,0
subject9959,0,0


In [74]:

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic:", topic_idx)
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]) )


        
def get_topics(docs, no_features=1000, no_topics=10, no_top_words=10):
    # NMF is able to use tf-idf
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
    tfidf = tfidf_vectorizer.fit_transform(docs)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()

    # Run NMF
    nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

    display_topics(nmf, tfidf_feature_names, no_top_words)
    

## Oversampling

In [75]:
def get_augmented_data(augmentation_method, n, ratio):
    dataset_key="erisk18_dev"
    data_cls=Dataset(key=dataset_key, remove_end=True)
    prefix=augmentation_method+str(n)+str(ratio)
    folder=augmentation_method+"/"+prefix
    truth_file=augmentation_method+"/"+prefix+".txt"
    data=data_cls.get_dataset( folder_name=folder, truth_name=truth_file,
                              return_id=True, partition="augmented", chunks=False)
    
    return data

In [76]:
over=get_augmented_data("Over", 8, 0)

In [77]:
svm= SVM_text(np.append(training[0], over[0]),np.append(training[1],over[1]), weighted=True)

In [79]:
svm.extract_features(test[0], feature="word", method="tf-idf", nrange1=(1,1) , nrange2=None,
                     k=None, stop_words="english", norm="l2", idf=False, feature_selection=False)

# Features:  (1150, 136468)
Training:  (1150, 136468)
Testing:  (820, 136468)


In [80]:
score= svm.train_and_test(test[1] , p_label="1")

Confusion matrix:  [[691  50]
 [ 33  46]]
Scores: [136468, 0.8987804878048781, 0.4791666666666667, 0.5822784810126582, 0.5257142857142857]


In [ ]:
#false_pos_all= pd.DataFrame(test[1],index=test[2], columns=["truth"])

In [81]:
false_pos_all["over"]=svm.predicted

In [82]:
false_pos_all

,truth,base,over
subject1010,0,0,0
subject1015,0,0,0
subject1044,0,0,0
subject1055,0,1,1
subject1073,0,0,0
...,...,...,...
subject9949,0,0,0
subject9951,1,0,0
subject9958,0,0,0
subject9959,0,0,0


## Thesaurus

In [83]:
thes=get_augmented_data("Thesaurus", 3, 0)

In [85]:
svm= SVM_text(np.append(training[0], thes[0]),np.append(training[1],thes[1]), weighted=True)

In [86]:
svm.extract_features(test[0], feature="word", method="tf-idf", nrange1=(1,1) , nrange2=None,
                     k=None, stop_words="english", norm="l2", idf=False, feature_selection=False)

# Features:  (735, 138175)
Training:  (735, 138175)
Testing:  (820, 138175)


In [88]:
score= svm.train_and_test(test[1] , p_label="1")

Confusion matrix:  [[700  41]
 [ 36  43]]
Scores: [138175, 0.9060975609756098, 0.5119047619047619, 0.5443037974683544, 0.5276073619631902]


In [89]:
false_pos_all["thes"]=svm.predicted

In [90]:
false_pos_all

,truth,base,over,thes
subject1010,0,0,0,0
subject1015,0,0,0,0
subject1044,0,0,0,0
subject1055,0,1,1,1
subject1073,0,0,0,0
...,...,...,...,...
subject9949,0,0,0,0
subject9951,1,0,0,0
subject9958,0,0,0,0
subject9959,0,0,0,0


## $Context_0$

In [91]:
a0=get_augmented_data("L0", 1, 20)

In [92]:
svm= SVM_text(np.append(training[0], a0[0]),np.append(training[1],a0[1]), weighted=True)

In [94]:
svm.extract_features(test[0], feature="word", method="tf-idf", nrange1=(1,1) , nrange2=None,
                     k=None, stop_words="english", norm="l2", idf=False, feature_selection=False)

# Features:  (889, 139372)
Training:  (889, 139372)
Testing:  (820, 139372)


In [95]:
score= svm.train_and_test(test[1] , p_label="1")

Confusion matrix:  [[734   7]
 [ 55  24]]
Scores: [139372, 0.9243902439024391, 0.7741935483870968, 0.3037974683544304, 0.43636363636363634]


In [96]:
false_pos_all["Context0"]=svm.predicted

In [97]:
false_pos_all

,truth,base,over,thes,Context0
subject1010,0,0,0,0,0
subject1015,0,0,0,0,0
subject1044,0,0,0,0,0
subject1055,0,1,1,1,0
subject1073,0,0,0,0,0
...,...,...,...,...,...
subject9949,0,0,0,0,0
subject9951,1,0,0,0,0
subject9958,0,0,0,0,0
subject9959,0,0,0,0,0


## $Context1$

In [98]:
a1=get_augmented_data("L1", 1, 30)

In [102]:
svm= SVM_text(np.append(training[0], a1[0]),np.append(training[1],a1[1]), weighted=True)

In [103]:
svm.extract_features(test[0], feature="word", method="tf-idf", nrange1=(1,1) , nrange2=None,
                     k=None, stop_words="english", norm="l2", idf=False, feature_selection=False)

# Features:  (569, 137819)
Training:  (569, 137819)
Testing:  (820, 137819)


In [104]:
score= svm.train_and_test(test[1] , p_label="1")

Confusion matrix:  [[711  30]
 [ 39  40]]
Scores: [137819, 0.9158536585365854, 0.5714285714285714, 0.5063291139240507, 0.5369127516778524]


In [105]:
false_pos_all["L1"]=svm.predicted

In [106]:
false_pos_all

,truth,base,over,thes,Context0,L1
subject1010,0,0,0,0,0,0
subject1015,0,0,0,0,0,0
subject1044,0,0,0,0,0,0
subject1055,0,1,1,1,0,0
subject1073,0,0,0,0,0,0
...,...,...,...,...,...,...
subject9949,0,0,0,0,0,0
subject9951,1,0,0,0,0,0
subject9958,0,0,0,0,0,0
subject9959,0,0,0,0,0,0


## Xi palabra relacionada

In [108]:
xi=get_augmented_data("Xi", 6, 10)

In [109]:
svm= SVM_text(np.append(training[0], xi[0]),np.append(training[1],xi[1]), weighted=True)

In [110]:
svm.extract_features(test[0], feature="word", method="tf-idf", nrange1=(1,1) , nrange2=None,
                     k=None, stop_words="english", norm="l2", idf=False, feature_selection=False)

# Features:  (984, 139604)
Training:  (984, 139604)
Testing:  (820, 139604)


In [111]:
score= svm.train_and_test(test[1] , p_label="1")

Confusion matrix:  [[713  28]
 [ 38  41]]
Scores: [139604, 0.9195121951219513, 0.5942028985507246, 0.5189873417721519, 0.5540540540540541]


In [112]:
false_pos_all["xi"]=svm.predicted

In [113]:
false_pos_all

,truth,base,over,thes,Context0,L1,xi
subject1010,0,0,0,0,0,0,0
subject1015,0,0,0,0,0,0,0
subject1044,0,0,0,0,0,0,0
subject1055,0,1,1,1,0,0,1
subject1073,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
subject9949,0,0,0,0,0,0,0
subject9951,1,0,0,0,0,0,0
subject9958,0,0,0,0,0,0,0
subject9959,0,0,0,0,0,0,0


## Resumen

In [115]:
false_pos_all.to_csv(r"D:/v1ktop/Drive/REPOS/augmentation_ap/predictionsDepSVM.csv")

In [ ]:
false_pos_all["truth"] = pd.to_numeric(false_pos_all["truth"])

In [ ]:
false_pos_all["random"] = pd.to_numeric(false_pos_all["random"])

In [ ]:
false_pos_all["thes"] = pd.to_numeric(false_pos_all["thes"])

In [ ]:
false_pos_all["a1"] = pd.to_numeric(false_pos_all["a1"])

In [ ]:
false_pos_all["a0"] = pd.to_numeric(false_pos_all["a0"])

In [ ]:
false_pos_all["xi"] = pd.to_numeric(false_pos_all["xi"])

In [ ]:
false_pos_all.columns=["truth", "over", "thes", "a0", "a1", "sum"]

In [ ]:
false_pos_all['sum'] = false_pos_all["a0"]+false_pos_all["a1"]-2

In [ ]:
false_pos_all[false_pos_all["truth"] == false_pos_all["sum"]]

In [ ]:
del false_pos_all["sum"]

In [ ]:
false_pos_all['sum'] = false_pos_all["over"]+false_pos_all["thes"]+false_pos_all["a0"]+false_pos_all["xi"]+false_pos_all["a1"]-5

In [ ]:
false_pos_all[false_pos_all['truth']==false_pos_all['sum']]

In [ ]:
false_pos_all[(false_pos_all["truth"]==1) & (false_pos_all["sum"]==-5) ]